In [14]:
from IGraph import *
from AbstractGraph import *
from UGraphModel import *
from DGraphModel import *
#from BayesNet import *

In [15]:
model = DGraphModel()
for idx in range(10):
    model.add(str(idx))

model.connect('0', '1')
model.connect('0', '5')
model.connect('1', '7')
model.connect('3', '2')
model.connect('3', '4')
model.connect('3', '7')
model.connect('3', '8')
model.connect('4', '8')
model.connect('6', '0')
model.connect('6', '1')
model.connect('6', '2')
model.connect('8', '2')
model.connect('8', '7')
model.connect('9', '4')
model.println()

TypeError: add() missing 1 required positional argument: 'tableProb'

In [16]:
DFS = 0
BFS = 1
sorter = TopoSorter(model)
topo = sorter.sort(BFS)
print("topoBFS:", topo)

topo = sorter.sort(DFS)
print("topoDFS:", topo)

topoBFS: []
topoDFS: []


In [17]:
import numpy as np
import time

In [532]:
class ConditionalProbabilityTable():
    def __init__(self,initTable,initParents,initNodeName):
        self.initTable = initTable
        TableShape = np.array(initTable).T.shape
        self.ProbabilityTable = np.array(initTable).T[:-1].reshape(TableShape[0]-1,TableShape[1])
        self.ProbabilityTableValues = np.array(initTable).T[-1].astype(np.float)
        self.ParentsList = initParents
        self.NodeName = initNodeName
        if self.isDiscreteDistribution() == False:
            self.ParentsNum = len(initParents)
            if self.ProbabilityTable.shape[0]-2 != self.ParentsNum:
                raise NameError("Init failed, number of parents is wrong!")
        
    def getValueList(self):
        return list(set(self.ProbabilityTable[-1]))

    def isDiscreteDistribution(self):
        if self.ParentsList == None:
            return True
        else:
            False
    
    def getNodeProbability(self,ParentState):
        RunTable = self.ProbabilityTable
        ValueTable = self.ProbabilityTableValues
        for name in self.ParentsList:
            if name in ParentState:
                State = np.array(ParentState[name])
                index = np.where( RunTable[0] == State )[0]
                RunTable = RunTable[1:,index] 
                ValueTable = ValueTable[index]
            else:
                raise NameError("Parents state is not enough")
        return RunTable[0], ValueTable
    
    def getOutput(self,ParentState):
        if self.isDiscreteDistribution(): 
            NodeProba = self.ProbabilityTable[0]
            ValueTable = self.ProbabilityTableValues
        else:
            NodeProba,ValueTable = self.getNodeProbability(ParentState)     
        RandomValue = np.random.random()
        for index in range(len(NodeProba)):
            if RandomValue < ValueTable[:index+1].sum():
                ParentState[self.NodeName] = NodeProba[index]
                return ParentState
        ParentState[self.NodeName] = NodeProba[-1]
        return ParentState

        
    def getOutputWithLikehood(self, ParentState, ParentLikelihoodWeight, Observations):
        if self.NodeName not in Observations.keys():
            if self.isDiscreteDistribution():   
                NodeProba = self.ProbabilityTable[0]
                ValueTable = self.ProbabilityTableValues
            else:
                NodeProba,ValueTable = self.getNodeProbability(ParentState)
            RandomValue = np.random.random()
            for index in range(len(NodeProba)):
                if RandomValue < ValueTable[:index+1].sum():
                    ParentState[self.NodeName] = NodeProba[index]    
                    return ParentState, ParentLikelihoodWeight
            ParentState[self.NodeName] = NodeProba[-1]
            return ParentState, ParentLikelihoodWeight
        else:
            if self.isDiscreteDistribution():   
                NodeProba = self.ProbabilityTable[0]
                ValueTable = self.ProbabilityTableValues
            else:
                NodeProba,ValueTable = self.getNodeProbability(ParentState)
            index = np.where(NodeProba==np.array(Observations[self.NodeName]))[0]
            ParentState[self.NodeName] = NodeProba[index]
            ParentLikelihoodWeight[self.NodeName] = float(ValueTable[index])
            return ParentState, ParentLikelihoodWeight

# Asia example

In [533]:
asia = ConditionalProbabilityTable( [ ['True', 0.5],[ 'False', 0.5 ]], None, 'asia' )

In [534]:
tuberculosis = ConditionalProbabilityTable(
    [[ 'True', 'True', 0.2 ],
     [ 'True', 'False', 0.8 ],
     [ 'False', 'True', 0.01 ],
     [ 'False', 'False', 0.99 ]], ['asia'],'tuberculosis')

In [535]:
smoking = ConditionalProbabilityTable( [ ['True', 0.5],[ 'False', 0.5 ]], None, 'smoking' )

In [536]:
lung = ConditionalProbabilityTable(
    [[ 'True', 'True', 0.75 ],
     [ 'True', 'False', 0.25 ],
     [ 'False', 'True', 0.02 ],
     [ 'False', 'False', 0.98 ]], ['smoking'], 'lung' )

bronchitis = ConditionalProbabilityTable(
    [[ 'True', 'True', 0.92 ],
     [ 'True', 'False', 0.08 ],
     [ 'False', 'True', 0.03 ],
     [ 'False', 'False', 0.97 ]], ['smoking'], 'bronchitis' )

tuberculosis_or_cancer = ConditionalProbabilityTable(
    [[ 'True', 'True', 'True', 1.0 ],
     [ 'True', 'True', 'False', 0.0 ],
     [ 'True', 'False', 'True', 1.0 ],
     [ 'True', 'False', 'False', 0.0 ],
     [ 'False', 'True', 'True', 1.0 ],
     [ 'False', 'True', 'False', 0.0 ],
     [ 'False', 'False', 'True', 0.0 ],
     [ 'False', 'False', 'False', 1.0 ]], ['tuberculosis', 'lung'], 'tuberculosis_or_cancer' )

In [537]:
xray = ConditionalProbabilityTable(
    [[ 'True', 'True', 0.885 ],
     [ 'True', 'False', 0.115 ],
     [ 'False', 'True', 0.04 ],
     [ 'False', 'False', 0.96 ]], ['tuberculosis_or_cancer'] , 'xray')

In [538]:
dyspnea = ConditionalProbabilityTable(
    [[ 'True', 'True', 'True', 0.96 ],
     [ 'True', 'True', 'False', 0.04 ],
     [ 'True', 'False', 'True', 0.89 ],
     [ 'True', 'False', 'False', 0.11 ],
     [ 'False', 'True', 'True', 0.96 ],
     [ 'False', 'True', 'False', 0.04 ],
     [ 'False', 'False', 'True', 0.89 ],
     [ 'False', 'False', 'False', 0.11 ]], ['tuberculosis_or_cancer', 'bronchitis'],'dyspnea')

In [539]:
NodeList = [asia,smoking,tuberculosis,lung,tuberculosis_or_cancer,bronchitis,xray,dyspnea]

# Normal Sampling

In [476]:
start_time = time.time()
RunForwardList = []
for i in range(100):
    RunningVar = {}
    RunningVar = asia.getOutput(RunningVar)
    RunningVar = smoking.getOutput(RunningVar)
    RunningVar = tuberculosis.getOutput(RunningVar)
    RunningVar = lung.getOutput(RunningVar)
    RunningVar = tuberculosis_or_cancer.getOutput(RunningVar)
    RunningVar = bronchitis.getOutput(RunningVar)
    RunningVar = xray.getOutput(RunningVar)
    RunningVar = dyspnea.getOutput(RunningVar)
    RunForwardList.append(RunningVar)
print("Infer Time: ", (time.time()-start_time))

Infer Time:  0.020061254501342773


In [467]:
Observations = { 'tuberculosis' : 'True', 'smoking' : 'False', 'bronchitis' : 'True' }

In [468]:
NodeNameList = ['asia','smoking','tuberculosis','lung','tuberculosis_or_cancer','bronchitis','xray','dyspnea']

In [469]:
RunValueList = []
KeyList = RunForwardList[0].keys()
for RunForwardValue in RunForwardList:
    OneRow = []
    ObservationsCheck = True
    for key in Observations:
        if Observations[key] != RunForwardValue[key]:
            ObservationsCheck = False
    if ObservationsCheck == True:
        for key in RunForwardValue: 
            OneRow.append(RunForwardValue[key])
        RunValueList.append(OneRow)
NumpyRunValue = np.array(RunValueList)
NumpyKey = np.array(list(KeyList))

In [470]:
for Node in NodeList:
    if Node.NodeName in Observations.keys():
        print(Node.NodeName, Observations[Node.NodeName])
    else:
        print(Node.NodeName)
        indexNumpyRunValue = np.where(NumpyKey==Node.NodeName)[0][0]
        for Value in Node.getValueList():
            print(Value,len(np.where(NumpyRunValue[:,indexNumpyRunValue] == Value)[0])/NumpyRunValue.shape[0])
    print('============================')

asia
True 0.9540918163672655
False 0.04590818363273453
smoking False
tuberculosis True
lung
True 0.017298735861610112
False 0.9827012641383899
tuberculosis_or_cancer
True 1.0
False 0.0
bronchitis True
xray
True 0.8855622089155023
False 0.11443779108449767
dyspnea
True 0.9680638722554891
False 0.031936127744510975


In [272]:
for Node in NodeList:
    if Node.NodeName in Observations.keys():
        print(Node.NodeName, Observations[Node.NodeName])
    else:
        print(Node.NodeName)
        indexNumpyRunValue = np.where(NumpyKey==Node.NodeName)[0][0]
        for Value in Node.getValueList():
            print(Value,len(np.where(NumpyRunValue[:,indexNumpyRunValue] == Value)[0])/NumpyRunValue.shape[0])
    print('============================')

asia
True 0.9629629629629629
False 0.037037037037037035
smoking False
tuberculosis True
lung
True 0.014814814814814815
False 0.9851851851851852
tuberculosis_or_cancer
True 1.0
False 0.0
bronchitis True
xray
True 0.9111111111111111
False 0.08888888888888889
dyspnea
True 0.9777777777777777
False 0.022222222222222223


# Likelihood

In [545]:
RunForwardList = []
RunForwardWeight = []
Observations = { 'tuberculosis' : 'True', 'smoking' : 'False', 'bronchitis' : 'True' }
start_time = time.time()
for i in range(1000000):
    w = 1
    RunningVar = {}
    ParentLikelihoodWeight = {}
    RunningVar,ParentLikelihoodWeight = asia.getOutputWithLikehood(RunningVar,ParentLikelihoodWeight,Observations)
    RunningVar,ParentLikelihoodWeight = smoking.getOutputWithLikehood(RunningVar,ParentLikelihoodWeight,Observations)
    RunningVar,ParentLikelihoodWeight = tuberculosis.getOutputWithLikehood(RunningVar,ParentLikelihoodWeight,Observations)
    RunningVar,ParentLikelihoodWeight = lung.getOutputWithLikehood(RunningVar,ParentLikelihoodWeight,Observations)
    RunningVar,ParentLikelihoodWeight = tuberculosis_or_cancer.getOutputWithLikehood(RunningVar,ParentLikelihoodWeight,Observations)
    RunningVar,ParentLikelihoodWeight = bronchitis.getOutputWithLikehood(RunningVar,ParentLikelihoodWeight,Observations)
    RunningVar,ParentLikelihoodWeight = xray.getOutputWithLikehood(RunningVar,ParentLikelihoodWeight,Observations)
    RunningVar,ParentLikelihoodWeight = dyspnea.getOutputWithLikehood(RunningVar,ParentLikelihoodWeight,Observations)
    RunForwardList.append(RunningVar)
    for key in Observations:
        w *= ParentLikelihoodWeight[key]
    RunForwardWeight.append(w)
print("Infer Time: ", (time.time()-start_time))

Infer Time:  94.96938180923462


In [546]:
RunValueList = []
WeightList = []
KeyList = []
for index_,RunForwardValue in enumerate(RunForwardList):
    OneRow = []
    for key in RunForwardValue: 
        if key not in Observations:
            if index_ == 0:
                KeyList.append(key)
            OneRow.append(RunForwardValue[key])
    RunValueList.append(OneRow)
    WeightList.append(RunForwardWeight[index_])
NumpyRunValue = np.array(RunValueList)
NumpyKey = np.array(list(KeyList))
NumpyWeight = np.array(WeightList)

In [547]:
TotalWeight = np.sum(NumpyWeight)
for Node in NodeList:
    if Node.NodeName in Observations.keys():
        print(Node.NodeName, Observations[Node.NodeName])
    else:
        print(Node.NodeName)
        indexNumpyRunValue = np.where(NumpyKey==Node.NodeName)[0][0]
        for Value in Node.getValueList():
            print(Value,np.sum(NumpyWeight[np.where(NumpyRunValue[:,indexNumpyRunValue] == Value)[0]]) / TotalWeight)
    print('============================')

asia
True 0.9524556359316124
False 0.04754436406838788
smoking False
tuberculosis True
lung
True 0.019756604314421587
False 0.980243395685578
tuberculosis_or_cancer
True 1.0
False 0.0
bronchitis True
xray
True 0.8840002900694606
False 0.11599970993053935
dyspnea
True 0.9598057752753468
False 0.04019422472465288


# Monty Hall  Example 

In [21]:
guest = ConditionalProbabilityTable( [ ['A', 1./3],[ 'B', 1./3 ],[ 'C', 1./3 ]], None, 'guest' )
prize = ConditionalProbabilityTable( [ ['A', 1./3],[ 'B', 1./3 ],[ 'C', 1./3 ]], None, 'prize' )

In [22]:
monty = ConditionalProbabilityTable( 
    [[ 'A', 'A', 'A', 0.0 ],
     [ 'A', 'A', 'B', 0.5 ],
     [ 'A', 'A', 'C', 0.5 ],
     [ 'A', 'B', 'A', 0.0 ],
     [ 'A', 'B', 'B', 0.0 ],
     [ 'A', 'B', 'C', 1.0 ],
     [ 'A', 'C', 'A', 0.0 ],
     [ 'A', 'C', 'B', 1.0 ],
     [ 'A', 'C', 'C', 0.0 ],
     [ 'B', 'A', 'A', 0.0 ],
     [ 'B', 'A', 'B', 0.0 ],
     [ 'B', 'A', 'C', 1.0 ],
     [ 'B', 'B', 'A', 0.5 ],
     [ 'B', 'B', 'B', 0.0 ],
     [ 'B', 'B', 'C', 0.5 ],
     [ 'B', 'C', 'A', 1.0 ],
     [ 'B', 'C', 'B', 0.0 ],
     [ 'B', 'C', 'C', 0.0 ],
     [ 'C', 'A', 'A', 0.0 ],
     [ 'C', 'A', 'B', 1.0 ],
     [ 'C', 'A', 'C', 0.0 ],
     [ 'C', 'B', 'A', 1.0 ],
     [ 'C', 'B', 'B', 0.0 ],
     [ 'C', 'B', 'C', 0.0 ],
     [ 'C', 'C', 'A', 0.5 ],
     [ 'C', 'C', 'B', 0.5 ],
     [ 'C', 'C', 'C', 0.0 ]], ['guest','prize'], 'monty' )

In [23]:
Observations = { 'guest' : 'A' }

In [24]:
NodeList = [guest,prize,monty]

In [25]:
RunForwardList = []
RunForwardWeight = []
for i in range(100000):
    w = 1
    RunningVar = {}
    ParentLikelihoodWeight = {}
    RunningVar,ParentLikelihoodWeight = guest.getOutputWithLikehood(RunningVar,ParentLikelihoodWeight,Observations)
    RunningVar,ParentLikelihoodWeight = prize.getOutputWithLikehood(RunningVar,ParentLikelihoodWeight,Observations)
    RunningVar,ParentLikelihoodWeight = monty.getOutputWithLikehood(RunningVar,ParentLikelihoodWeight,Observations)
    RunForwardList.append(RunningVar)
    for key in Observations:
        w *= ParentLikelihoodWeight[key]
    RunForwardWeight.append(w)

NameError: name 'random' is not defined

In [26]:
RunValueList = []
WeightList = []
KeyList = []
for index_,RunForwardValue in enumerate(RunForwardList):
    OneRow = []
    for key in RunForwardValue: 
        if key not in Observations:
            if index_ == 0:
                KeyList.append(key)
            OneRow.append(RunForwardValue[key])
    RunValueList.append(OneRow)
    WeightList.append(RunForwardWeight[index_])
NumpyRunValue = np.array(RunValueList)
NumpyKey = np.array(list(KeyList))
NumpyWeight = np.array(WeightList)

In [27]:
TotalWeight = np.sum(NumpyWeight)
for Node in NodeList:
    if Node.NodeName in Observations.keys():
        print(Node.NodeName, Observations[Node.NodeName])
    else:
        print(Node.NodeName)
        indexNumpyRunValue = np.where(NumpyKey==Node.NodeName)[0][0]
        for Value in Node.getValueList():
            print(Value,np.sum(NumpyWeight[np.where(NumpyRunValue[:,indexNumpyRunValue] == Value)[0]]) / TotalWeight)
    print('============================')

guest A
prize


IndexError: index 0 is out of bounds for axis 0 with size 0

In [35]:
a = ['i']
b = ['']
print(b[0])
if not a:
    print("List a is empty")
else:
    print("List a is not empty")

if b == ['']:
    print("List b is empty")
else:
    print("List b is not empty")



List a is not empty
List b is empty


In [14]:
file = open("test.txt")
for line in file:
    ObservationList = dict()
    Probability = dict()
    print("line: ",line)
    if len(line) == 2:
        numVertex = line.split('\n')[0]
    else:
        inputGraph = line.split('\n')[0]
        input = inputGraph.split(';')
        print("input: ", input)
        prob = str(input[0])
        print("prob: ", prob)
        for subProb in prob:
            print("subProb: ",subProb)
            subP = subProb.split('=')
            print("subP: ", subP)
            Probability.update({subP[0]:subP[1]})
        condition = input[1].split(',')
        print("condition: ", condition)
        for subcondition in condition:
            sub = subcondition.split('=')
            print("subcondition: ",sub)
            ObservationList.update({sub[0]:sub[1]})
        print("Probability: ",Probability)
        print("ObservationList: ",ObservationList)
        print("=======")


line:  2

line:  G=A;I=Cao,D=Kho

input:  ['G=A', 'I=Cao,D=Kho']
prob:  G=A
subProb:  G
subP:  ['G']


IndexError: list index out of range